### Задача
**Даны:** твиты на русском языке с присписанной им тональностью (позитивная или негативная оценка содержится в твите)

**Нужно:** построить классификатор, распознающий тональность твита

In [2]:
import string
import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from nltk.corpus import stopwords
from string import punctuation
import re
morph = MorphAnalyzer()

In [3]:
import nltk

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [50]:
# https://raw.githubusercontent.com/stopwords-iso/stopwords-ru/master/raw/stop-words-russian.txt

with open('stopwords.txt', 'r', encoding='utf-8') as f:
    f = f.readlines()
    f[0] = f[0].strip('\ufeff')
    stop_words = []
    for i in f:
        stop_words.append(i.strip())

In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [7]:
train_data = pd.read_csv('data/sentiment_twitter/train_sentiment_ttk.tsv', sep='\t')
test_data = pd.read_csv('data/sentiment_twitter/test_sentiment_ttk.tsv', sep='\t')

In [8]:
train_data.head(10)

,label,text
0,0,"@mkomov Максим, Вашем письмо мы получили. Наши..."
1,0,«Мегафон» стал владельцем 50% акций «Евросети»
2,-1,RT @fuckkiev: “@EvaKobb: МТС Россия прислала ж...
3,1,ВИДЕО: http://t.co/PSMLAhR4fI Реклама со смехо...
4,-1,"@parfenov1960 потому что МТС достало, а пчел н..."
5,-1,RT @f_u_c_k_y_o_u_: Билайн интернет стал полны...
6,0,На Дне города в Тюмени «МегаФон» разыграет сма...
7,0,&amp;laquo;Ростелеком&amp;raquo; стал партнеро...
8,0,RT @parfenov1960: Сотовый оператор Мегафон про...
9,1,«МегаФон» поможет контролировать расходы на Ин...


In [9]:
test_data.head(10)

,label,text
0,-1,RT vzglyad: По делу о работе МТС в Узбекистане...
1,0,RT @kevinuyatukox: http://t.co/ljtrjq91v3 #Кре...
2,0,#Оформить кредитную карту в банке мтс http://t...
3,0,#Как перевести деньги с билайна на кредитную к...
4,0,#Начальник отдела кредитного контроля оао мтс ...
5,-1,На МТС завели дело за обман абонентов в Крыму ...
6,-1,ФАС возбудила дело на МТС за ложь в рекламе\nh...
7,0,RT @kennethenozacuk: http://t.co/ds5kOamzx6 #М...
8,-1,RT @vishtorskayaa: Отказано в доступе при уста...
9,0,RT @Serg_58: Tele2 хочет продавать смартфоны п...


### Baseline

##### CountVectorizer + LogisticRegression

In [10]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit(train_data.text.values) 

X_train = count_vectorizer.transform(train_data.text.values)
X_test = count_vectorizer.transform(test_data.text.values)

y_train = train_data.label.values
y_test = test_data.label.values

In [11]:
clf = LogisticRegression(penalty="l1", C=0.1)
clf.fit(X_train, y_train)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [12]:
y_pred = clf.predict(X_test)

In [13]:
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ',f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ',f1_score(y_test, y_pred, average='micro'))

             precision    recall  f1-score   support

         -1       0.69      0.59      0.64       902
          0       0.61      0.80      0.69       972
          1       0.30      0.03      0.06       180

avg / total       0.62      0.64      0.61      2054

Макросредняя F1 мера -  0.463064212113
Микросредняя F1 мера -  0.638753651412


##### TfidfVectorizer + LogisticRegression

In [14]:
tfidf = TfidfVectorizer()
tfidf.fit(train_data.text.values)
X_train = tfidf.transform(train_data.text.values)
X_test = tfidf.transform(test_data.text.values)

In [15]:
clf = LogisticRegression(penalty='l1')
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [16]:
y_pred = clf.predict(X_test)

In [17]:
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ',f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ',f1_score(y_test, y_pred, average='micro'))

             precision    recall  f1-score   support

         -1       0.70      0.69      0.70       902
          0       0.66      0.76      0.71       972
          1       0.37      0.09      0.15       180

avg / total       0.65      0.67      0.65      2054

Макросредняя F1 мера -  0.517616688856
Микросредняя F1 мера -  0.671372930867


Очевидно, что использование TfidfVectorizer'a дает более высокий результат, значит, и далее будем использовать его.

### Улучшение качества классификации

##### TfidfVectorizer + LogisticRegression

Очистка данных

In [9]:
def clean_data(text):
    text = re.sub('http[^ ]*?($| |,)', '', text)
    text = re.sub('@', '', text)
    text = re.sub('[A-Za-z0-9]+', '', text)
    text = re.sub(r'[^\w\s]', '', text)

    return text

Очистка данных и учёт списка стоп-слов дает более низкий результат:
##### Очистка данных + гиперпараметры:

In [33]:
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ', f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ', f1_score(y_test, y_pred, average='micro'))

             precision    recall  f1-score   support

         -1       0.72      0.71      0.72       902
          0       0.69      0.76      0.72       972
          1       0.50      0.28      0.36       180

avg / total       0.69      0.70      0.69      2054

Макросредняя F1 мера -  0.600668936986
Микросредняя F1 мера -  0.695715676728


##### Стоп-слова + гиперпараметры:

In [43]:
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ', f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ', f1_score(y_test, y_pred, average='micro'))

             precision    recall  f1-score   support

         -1       0.75      0.63      0.68       902
          0       0.65      0.80      0.72       972
          1       0.44      0.28      0.34       180

avg / total       0.68      0.68      0.67      2054

Макросредняя F1 мера -  0.580613304041
Микросредняя F1 мера -  0.675754625122


##### Очистка данных + стоп-слова + гиперпараметры:

In [53]:
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ', f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ', f1_score(y_test, y_pred, average='micro'))

             precision    recall  f1-score   support

         -1       0.73      0.61      0.66       902
          0       0.65      0.78      0.71       972
          1       0.46      0.32      0.38       180

avg / total       0.67      0.67      0.66      2054

Макросредняя F1 мера -  0.583832099676
Микросредняя F1 мера -  0.666017526777


In [18]:
def normalize(text):
    """
    функция нормализации
    
    ::парметры::
    @text - ненормализованный текст (string)
    
    ::returns::
    нормализованный текст (string)
    """
#     text = clean_data(text)

    tokens = word_tokenize(text)
    lemmas = [morph.parse(token)[0].normal_form for token in tokens]
    
#     analized = [morph.parse(token)[0].normal_form for token in tokens]
#     lemmas = [lemma for lemma in analized
#               if lemma not in stop_words]
    
    return ' '.join(lemmas)

In [19]:
train_data['normalized'] = train_data['text'].apply(normalize)
test_data['normalized'] = test_data['text'].apply(normalize)

In [20]:
tfidf = TfidfVectorizer()
tfidf.fit(train_data['normalized'].values)

X_train = tfidf.transform(train_data['normalized'].values)
X_test = tfidf.transform(test_data['normalized'].values)

Воспользуемся перебором гиперпараметров по сетке:

##### До перебора (без очистки данных и стоп-слов):

In [21]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [22]:
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ', f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ', f1_score(y_test, y_pred, average='micro'))

             precision    recall  f1-score   support

         -1       0.70      0.71      0.71       902
          0       0.67      0.76      0.72       972
          1       0.58      0.12      0.20       180

avg / total       0.68      0.68      0.67      2054

Макросредняя F1 мера -  0.541085803258
Микросредняя F1 мера -  0.684031158715


##### Перебор гиперпараметров по сетке:

In [59]:
from sklearn.model_selection import GridSearchCV

In [60]:
params = {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}

In [61]:
def get_fitted_searcher(X_train, y_train):
    searcher = GridSearchCV(LogisticRegression(), params)
    searcher.fit(X_train, y_train)
    return searcher

In [62]:
searcher = get_fitted_searcher(X_train, y_train)

In [63]:
searcher.best_params_

{'C': 10, 'penalty': 'l2'}

In [64]:
clf = LogisticRegression(C=10, penalty='l2')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

##### TfidfVectorizer + LogisticRegression(C=10, penalty='l2')

In [65]:
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ', f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ', f1_score(y_test, y_pred, average='micro'))

             precision    recall  f1-score   support

         -1       0.72      0.74      0.73       902
          0       0.70      0.75      0.73       972
          1       0.51      0.27      0.35       180

avg / total       0.70      0.70      0.70      2054

Макросредняя F1 мера -  0.603415121571
Микросредняя F1 мера -  0.704479065239


In [23]:
def print_important(vectorizer, clf, topn=10):
    features = vectorizer.get_feature_names()
    classes = clf.classes_
    importances = clf.coef_
    for i, cls in enumerate(classes):
        print('Значимые слова для класса - ', cls)
        important_words = sorted(list(zip(features, importances[i])), key=lambda x: abs(x[1]), reverse=True)[:topn]
        print([word for word,_ in important_words])
        print()

In [24]:
print_important(tfidf, clf, topn=10)

Значимые слова для класса -  -1
['не', 'крым', 'нет', 'tele2', 'http', 'отключить', 'спасибо', 'co', 'связь', 'оштрафовать']

Значимые слова для класса -  0
['не', 'для', 'доллар', 'пожалуйста', 'http', 'связь', 'co', 'работать', 'beeline_rus', 'настройка']

Значимые слова для класса -  1
['спасибо', 'узбекистан', 'любить', 'не', 'хороший', '4g', 'запустить', 'бесплатный', 'защита', 'lte']



Попробуем добавить в TfidfVectorizer параметр ngram_range.

In [27]:
tfidf_r = TfidfVectorizer(ngram_range=(1, 5))
tfidf_r.fit(train_data['normalized'].values)

X_train = tfidf_r.transform(train_data['normalized'].values)
X_test = tfidf_r.transform(test_data['normalized'].values)

In [28]:
clf = LogisticRegression(C=10, penalty='l2')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [29]:
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ', f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ', f1_score(y_test, y_pred, average='micro'))

             precision    recall  f1-score   support

         -1       0.67      0.79      0.72       902
          0       0.71      0.68      0.70       972
          1       0.55      0.17      0.26       180

avg / total       0.68      0.68      0.67      2054

Макросредняя F1 мера -  0.557303532116
Микросредняя F1 мера -  0.682570593963


Результат не улучшился.

Попробуем заменить лемматизацию на стемминг.

In [43]:
from nltk.stem.snowball import SnowballStemmer 
stemmer = SnowballStemmer('russian')

In [44]:
def normalize(text):
    tokens = word_tokenize(text)
    lemmas = [stemmer.stem(token) for token in tokens]
    
    return ' '.join(lemmas)

In [45]:
train_data['normalized'] = train_data['text'].apply(normalize)
test_data['normalized'] = test_data['text'].apply(normalize)

In [46]:
tfidf = TfidfVectorizer()
tfidf.fit(train_data['normalized'].values)

X_train = tfidf.transform(train_data['normalized'].values)
X_test = tfidf.transform(test_data['normalized'].values)

In [47]:
clf = LogisticRegression(C=10, penalty='l2')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [48]:
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ', f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ', f1_score(y_test, y_pred, average='micro'))

             precision    recall  f1-score   support

         -1       0.72      0.76      0.74       902
          0       0.71      0.74      0.72       972
          1       0.51      0.26      0.34       180

avg / total       0.70      0.71      0.70      2054

Макросредняя F1 мера -  0.602751632321
Микросредняя F1 мера -  0.70593962999


Таким образом, значительного прироста в качестве не произошло.